In [1]:
import numpy as np
import math
import pandas as pd
#from tqdm import tqdm 

In [2]:
def GPS2CartesianCoordinate(lati, long, radius):
    rad_lati = math.radians(lati)
    rad_long = math.radians(long)
    x = radius * math.cos(rad_lati) * math.cos(rad_long)
    y = radius * math.cos(rad_lati) * math.sin(rad_long)
    z = radius * math.sin(rad_lati)
    return x, y, z

def ArcDistance(p1, p2, radius):
    psi = np.arccos(p1.dot(p2.T)/(radius*radius))
    return psi * radius

In [3]:
earth_radius = 6371.0
def dist(lat1, lon1, lat2, lon2):
    # GPS coordinate

    # 토트넘 스타디움 : 51.60440517547472, -0.06560569079349023
    # 레알마드리드 스타디움 : 40.459278005696014, -3.694492799000267
    x1, y1, z1 = GPS2CartesianCoordinate(lat1, lon1, earth_radius)
    x2, y2, z2 = GPS2CartesianCoordinate(lat2, lon2, earth_radius)

    p1 = np.array([x1, y1, z1])
    p2 = np.array([x2, y2, z2])

    # Arc distance between Boston and New York
    return ArcDistance(p1, p2, earth_radius)




In [4]:
dist(50.083, 14.417, 50.084, 14.417)

0.11119493076987927

In [5]:
loc = pd.read_csv('./data/lat_lon.csv')
alti = pd.read_csv('./data/elevation/seoul_elevation.csv')

In [6]:
alti

,고도,좌표정보(x),좌표정보(y),경도,위도
0,16.78,200305.31,547459.91,127.003454,37.526632
1,9.62,200305.31,547369.91,127.003454,37.525821
2,5.69,200305.31,547279.91,127.003454,37.525010
3,5.38,200305.31,547189.91,127.003454,37.524199
4,19.62,200395.31,547549.91,127.004472,37.527443
...,...,...,...,...,...
111532,132.81,210697.72,554846.65,127.121131,37.593124
111533,121.28,210697.72,554756.65,127.121129,37.592313
111534,113.31,210697.72,554666.65,127.121128,37.591502
111535,146.72,210787.72,555026.65,127.122152,37.594745


In [13]:
loc

,0,x,y
0,서울특별시 성북구 석관동 245-7,127.057876,37.610424
1,서울특별시 강서구 공항동 676-5,126.815828,37.555039
2,서울특별시 마포구 망원동 384-6,126.908425,37.554076
3,서울특별시 강서구 화곡동 1045-75,126.836016,37.541445
4,서울특별시 양천구 목동 527-6,126.873370,37.545846
...,...,...,...
48417,서울특별시 마포구 연남동 561-70,126.921294,37.561088
48418,서울특별시 용산구 원효로2가 80-28,126.962073,37.536785
48419,서울특별시 송파구 석촌동 245-31,127.100702,37.501366
48420,서울특별시 용산구 이태원동 225-46,126.989803,37.539926


In [15]:
print(max(alti['경도']))
print(max(loc['x']))
print(min(alti['경도']))
print(min(loc['x']))



127.18876911535796
127.174326651386
126.75937074015394
126.79990309262


In [52]:
alti_sum_list = []



#for i in tqdm(range(len(loc))):
for i in range(len(loc)):
    
    min_d1 = 1000
    min_d2 = 1000
    min_d3 = 1000
    min_d4 = 1000
    min_i1 = 0
    min_i2 = 0
    min_i3 = 0
    min_i4 = 0
    zero_tmp = 0
    
    for j in range(len(alti)):
        if abs(loc['y'][i] - alti['위도'][j])<0.01 and abs(loc['x'][i] - alti['경도'][j])<0.01:
            dist_tmp = dist(loc['y'][i], loc['x'][i], alti['위도'][j], alti['경도'][j])
            if dist_tmp < min_d1:
                min_d2 = min_d1
                min_i2 = min_i1
                min_d1 = dist_tmp
                min_i1 = j
                
            elif min_d1 < dist_tmp < min_d2 :
                min_d3 = min_d2
                min_i3 = min_i2
                min_d2 = dist_tmp
                min_i2 = j
                
            elif min_d2 < dist_tmp < min_d3 :
                min_d4 = min_d3
                min_i4 = min_i3
                min_d3 = dist_tmp
                min_i3 = j
                
            elif min_d3 < dist_tmp < min_d4 :
                min_d4 = dist_tmp
                min_i4 = j
                
    alti_sum = 0
    for zero in [min_i1,min_i2,min_i3,min_i4]:
        if zero != 0:
            zero_tmp+=1
    
    alti_sum += abs(alti['고도'][min_i1]-alti['고도'][min_i2])
    alti_sum += abs(alti['고도'][min_i1]-alti['고도'][min_i3])
    alti_sum += abs(alti['고도'][min_i1]-alti['고도'][min_i4])
    print(alti_sum/zero_tmp)
    print(alti_sum)
    print(min_i1, min_i2, min_i3, min_i4)
    print(min_d1, min_d2, min_d3, min_d4)
    
    alti_sum_list.append(alti_sum/zero_tmp)
loc['alti'] = alti_sum_list
loc.to_csv('./data/lat_lon_alti.csv',index=False)

0.3274999999999997
1.3099999999999987
78139 78174 78140 78175
0.04495424442503048 0.048222552577043394 0.08892133827692222 0.09061801704569863
0.18000000000000016
0.7200000000000006
18825 18904 18826 18905
0.044201124378975096 0.05604954585746849 0.07809240503846793 0.0853579730503998
0.05749999999999966
0.22999999999999865
56755 56756 56702 56807
0.03573592109900496 0.0623951006295756 0.07757155005757116 0.11251811455248069
0.11750000000000016
0.47000000000000064
20290 85400 85399 85453
0.0198866223854043 0.03633765569163245 0.053830071979248835 0.09680570534954815
0.7125000000000004
2.8500000000000014
22652 87217 22651 87218
0.030518117711307358 0.03464110841421826 0.06148634654922683 0.07515368647932165
0.11250000000000071
0.45000000000000284
78322 40101 78321 78295
0.03680050221027174 0.043018786121791434 0.06282385932757313 0.07598602508869669
1.3125000000000004
5.250000000000002
81379 81380 81474 81285
0.011797214932856001 0.0786186939679364 0.08830554617847851 0.0927581991259503

2.0625
8.25
60274 60208 60275 60273
0.04454114897624348 0.04541738786830922 0.09804881541197182 0.1030269766421004
0.11000000000000121
0.44000000000000483
48814 77248 77296 48815
0.02516304799773998 0.03725409153475104 0.05495376279993533 0.07303566894209154
1.0699999999999985
4.279999999999994
14114 45070 45069 44993
0.03398444994649665 0.03593310433748545 0.05944094528837522 0.08159941520073169
0.54
2.16
48559 71294 108339 108340
0.01858720643594879 0.0334864667037861 0.03983833967816028 0.05309676134503039
6.287500000000001
25.150000000000006
106563 94204 106592 94203
0.03159049606214963 0.0511551521693958 0.06566771388961086 0.07689132067942736
7.035
28.14
60400 60399 60470 60469
0.024454639895465462 0.07062790593888181 0.07957769789408682 0.10355169621405039
0.24750000000000005
0.9900000000000002
28551 73338 28550 73339
0.03079754327656471 0.05395215664882278 0.06122833624173615 0.06735684479281988
0.44249999999999945
1.7699999999999978
79882 5517 79940 79883
0.027341036175791564 

0.1200000000000001
0.4800000000000004
98178 98266 98177 98179
0.0609935462794532 0.06327093967110772 0.06408420928262328 0.13997764618783615
1.3899999999999997
5.559999999999999
86270 86271 86230 86330
0.02856890052712836 0.06532210805798544 0.08204667443296744 0.10499637945257959
0.18249999999999966
0.7299999999999986
50348 50347 50414 50413
0.0435488193878439 0.06028742745981052 0.07358692090496806 0.08457552270508295
1.5999999999999996
6.399999999999999
98715 98714 98807 98806
0.04292608441315454 0.051304426355838186 0.08635470422257228 0.09081155507187452
0.6775000000000002
2.710000000000001
66788 1363 66708 66789
0.03098208758353424 0.03860471256794185 0.06819414643170799 0.07505972745966516
3.147500000000001
12.590000000000003
63690 63689 63766 63691
0.015969178190599856 0.07860912108986093 0.10079857807787311 0.10291498622199768
0.35749999999999993
1.4299999999999997
80680 80681 80601 80762
0.041449342530802895 0.048797020202393473 0.09718500069041884 0.10057237743106465
2.94999

0.23999999999999977
0.9599999999999991
80527 80604 80526 80528
0.03244315711330631 0.058381503339630686 0.08945021210094699 0.10180605687642401
1.2850000000000006
5.140000000000002
85708 85707 85746 85745
0.04375724320818413 0.047460718200375854 0.09347950555906173 0.0952706342709748
3.682500000000001
14.730000000000004
53685 27560 27608 27607
0.0029368629922927234 0.04676071807041957 0.07492494361689293 0.08192855710922686
1.125
4.5
46927 47014 46926 47013
0.01650837209319389 0.07485995891216128 0.08510365693860632 0.11213406004400854
3.744999999999999
14.979999999999997
109890 109889 109958 109957
0.024260380583926943 0.06592817293585798 0.09384151317848355 0.11208932275124386
2.8625
11.45
37020 37021 37082 36959
0.006565652728853096 0.08473420756538222 0.08639232087318215 0.09352177343773185
2.1449999999999987
8.579999999999995
34958 35009 34959 35010
0.02999279376200001 0.06051535128557053 0.08979639724823998 0.10404687774361668
0.24249999999999972
0.9699999999999989
77303 48860 77

0.032500000000000195
0.13000000000000078
57006 57007 57055 57056
0.03281700552479 0.06306133888948467 0.07988510584686266 0.09633963687263278
5.3575000000000035
21.430000000000014
27506 53586 27454 53587
0.042868201728023185 0.0433849118277346 0.04733702816342316 0.05688177334657373
2.9499999999999993
11.799999999999997
32383 85092 32334 85156
0.023480706345281238 0.06233729552952098 0.06939133886971263 0.08548426751706054
4.354999999999999
17.419999999999995
75070 105216 75014 105215
0.037102944434533464 0.05000801864914653 0.05366288572147337 0.07564478855860879
0.4950000000000001
1.9800000000000004
105833 107968 108006 107967
0.010088187664413398 0.04901971952833221 0.06256970796976567 0.0680219856667494
3.224999999999998
12.899999999999991
102596 62121 102595 62120
0.033606559118108434 0.03952274009983889 0.057076171754922564 0.08755036945803683
2.3049999999999997
9.219999999999999
64814 64723 64815 64907
0.019980892129813053 0.07160337120113817 0.08484140834178319 0.10860519251025

...

3.000000000000001
12.000000000000004
22070 86421 86358 86422
0.026374590896512513 0.05767614307492085 0.06031731918812313 0.06750420407469569
2.7399999999999993
10.959999999999997
37228 37164 37163 37229
0.04131049249241505 0.06436669210904074 0.08612128170332572 0.12121743175262298
7.052499999999998
28.209999999999994
94609 106716 94542 106748
0.03683611898356013 0.037963318760409905 0.05312586399424967 0.05524296327370915
0.6874999999999991
2.7499999999999964
98441 98442 98351 98531
0.045366657641506464 0.054947776278782716 0.07870611436162395 0.11851191940332408
2.55
10.2
59960 59903 59961 59959
0.05359162737963375 0.05711910623313909 0.07187868584500383 0.12976102978474457
0.3500000000000001
1.4000000000000004
65249 306 65341 307
0.018530874080490586 0.05213918819732769 0.07490292733826728 0.07502778101592476
6.982499999999998
27.929999999999993
99911 60315 99912 99824
0.022688940377833453 0.03690321991371713 0.07431627051891382 0.07709096933082188
1.6825
6.73
108379 71477 71420 10

1.12
4.48
29670 29669 29741 29740
0.026343989252298427 0.0674238479817467 0.08184848792607985 0.10271814983180866
0.49499999999999966
1.9799999999999986
65783 547 65784 546
0.04115508128054592 0.04578756092474388 0.0492856853119352 0.06904270304526693
1.21
4.84
83565 83566 83496 83630
0.030654547441479957 0.0692873550718396 0.07419246682654633 0.11179662273344
2.3125
9.25
23734 23733 23797 23735
0.0212135254719699 0.07292081524318433 0.10247160562393808 0.10882774041795269
0.9199999999999999
3.6799999999999997
71243 108303 108326 71244
0.04657986632798458 0.04874379816153032 0.0498701364256306 0.05176081047826418
0.15749999999999975
0.629999999999999
56855 56856 56906 56907
0.026797999860747088 0.06762892381928225 0.08084040250614487 0.10193425357859291
0.4275000000000002
1.7100000000000009
98635 98543 98636 98634
0.0396156705990467 0.06580324290443061 0.07025109416015582 0.12026526299294055
13.285
53.14
96035 96071 67 96072
0.040160772983372034 0.049943005163151685 0.06017153164655751

6.735000000000001
26.940000000000005
32775 32774 32825 32824
0.023537445334405036 0.06801748061868401 0.08588370934848837 0.10699822994612697
1.8549999999999986
7.419999999999995
51720 91372 91310 91371
0.02974960819641806 0.052014147871231764 0.05716681718363055 0.07241547290062185
1.3125000000000004
5.250000000000002
81379 81380 81474 81285
0.021966600181092998 0.0740876184505752 0.07822144316913872 0.1047381177329451
1.7249999999999996
6.899999999999999
84316 84317 84346 84284
0.028648289365547062 0.06234387356122203 0.08868415298506699 0.09949966542581853
0.6574999999999998
2.629999999999999
29526 79122 29527 79163
0.03116481616569714 0.06310205787207304 0.0697086950647482 0.0759596890803589
3.7775
15.11
21650 21716 21649 86148
0.026146187616624114 0.06557940455203083 0.08540700423036819 0.09756360666894963
2.9850000000000003
11.940000000000001
86058 21526 86059 86057
0.01589252048783818 0.04066645214546922 0.09150827110480309 0.09160739751865976
0.19750000000000023
0.7900000000000

0.2599999999999998
1.0399999999999991
20584 20585 20504 20664
0.04377855598710137 0.06245205850407251 0.07102403983689488 0.12211539592265883
0.26750000000000007
1.0700000000000003
35442 89291 89290 89213
0.022970615006576496 0.0393058503611346 0.05092011152113147 0.09654116973826335
0.08250000000000002
0.33000000000000007
79408 79409 79363 79455
0.0603953415159477 0.06255602241487027 0.0648583893124119 0.13860820099870402
3.0275000000000016
12.110000000000007
75300 105379 105378 105417
0.008991831214266712 0.03966705310826293 0.05054214748969925 0.09690090089633237
11.037499999999998
44.14999999999999
61344 61345 61421 101028
0.03795583974204883 0.063279984364225 0.07440450638748516 0.08324239617313865
5.062499999999998
20.249999999999993
97822 97909 97821 97908
0.024941257228140445 0.0648463681730286 0.09341715326695614 0.11094983931026366
0.04749999999999943
0.18999999999999773
79603 79654 79602 79604
0.04492075073302086 0.046276259439917936 0.09331835567290034 0.10764817212606267
2

KeyboardInterrupt: 